# Using Python to call 3E's Rest API
**NOTE:** The following assumes decent usage of Python. 

At time of writing there isn't much documentation on how to use the Rest API for 3e. Below are a few samples of some calls to create, retrieve and update. The list of examples will grow within this notebook as time permits.

There are a couple of articles in the Thomson Reuters Knowledge base that cover a few of the basics. Specifically:
- TR-19784 : How to obtain access to 3E APIs 
- TR-19969 : ASG - 3E OData - Troubleshooting Guide (E) 
- TR-19847 : 3E API & OData - A Primer for Developers 
- TR-19363 : Using Postman to Demo 3E API Connections 

For this sample, we'll use NTLM authentication, simply because that's all that is available currently. If you've got a cloud instance, you'd go through the authentication to get your token. See the section in TR-19847 for more information on this.

Because this will be saved to the internet, credentials and instance urls are tucked into another Python file in the same folder as this one. Use whatever method you like to accomplish this hiding of credentials, whether that be environment variables, calls out to third party password managers, or read from a database. For simplicity, I'm using a service account with limited permissons. The `environment.py` file looks a little like the following:

```python
class Credentials:
    
    def __init__(self, wapi=None, instance=None, run_as=None, run_as_password=None):
        self.wapi = wapi
        self.instance = instance
        self.run_as = run_as
        self.run_as_password = run_as_password
        
    def load_defaults(self):
        self.wapi = 'your_internal_url.local'
        self.instance = 'te_3e_dbname'
        self.run_as = 'domain\some_user_name'
        self.run_as_password = 'some_password'
    
    def base_url(self, version=1):
        return f"https://{self.wapi}/{self.instance}/web/api/v{version}"
```

#### Requirements
`Requests` is the only 3rd party library used here. The rest is Python standard library.

The following uses `requests_ntlm` which can be installed in your environment with `pip install requests-ntlm`

In [ ]:
from operator import attrgetter  # allows us to use a dotted notation for the API responses
import requests
from requests_ntlm import HttpNtlmAuth
import json
import warnings
from types import SimpleNamespace
            
from environment import Credentials

In [ ]:
def get_value(dotted, obj, default=None):
    """ 
    Convenience function to return a default value when AttributeError is encountered
    """
    try:
        return attrgetter(dotted)(obj)
    except AttributeError:
        return default

In [ ]:
# Load up the default connection information
env = Credentials()
env.load_defaults()

In [ ]:
# Fire up a NTLM session with the Wapi
# working with a self signed certificate here that can be problematic
# for now, turning off the ssl verification for local 3e endpoint

warnings.filterwarnings("ignore", message="Unverified HTTPS request")

session = requests.Session()
session.auth = HttpNtlmAuth(env.run_as, env.run_as_password)
session.verify = False


## Get details about a Timekeeper
This one is an easy place to start. This endpoint `/api/v1/timekeeper` will per the documentation, "Gets Timekeepers and returns a TimekeeperGetResponse."

In [ ]:
# specify the url of the REST API endpoint
timekeeper_url = f"{env.base_url()}/timekeeper"

# issue a GET request with the query parameters and store the response
query_params = {'Number': "3171", }
response = session.get(timekeeper_url, params=query_params)

# decode the response data from JSON to a Python dotted notation object
# the reponse has 'timekeepers' as top level key
# because by definition, everything coming back is JSON serialized so 
# the object_hook with a SimpleNamespace can give us dot notation

# it is also possible to keep it as a normal dictionary by removing the object_hook

timekeepers = response.json(object_hook=lambda d: SimpleNamespace(**d)).timekeepers

In [ ]:
# if you want to see what the shape of timekeepers 'looks like' uncomment the following
# timekeepers

In [ ]:
# couple of ways to get the value
for timekeeper in timekeepers:
    print(timekeeper.id)
    print(timekeeper.attributes.Number.value)
    print(timekeeper.attributes.DisplayName.value)
    print(attrgetter("attributes.DisplayName.value")(timekeeper))
    # demonstrate why get_value may be of some use
    print(get_value("attributes.DisplayName.value.some_non_existant", timekeeper, "Unknown"))
    print(get_value("attributes.DisplayName.value.some_non_existant", timekeeper))

### Advanced Filtering
It looks like there is some ability to specify the attributes and child objects to return. On a large list, I suspect this would improve performance. It feels a little bit like using GraphQL queries, but not as easy.

I've not yet discovered how to return a `TimekeeperGetResponse` without any child objects. Passing an empty list, empty string, and `None` all don't perform as expected.

In [ ]:
query_params = {'Number': "3171",
                'AdvancedFilter.AttributesToInclude': ["Number", "DisplayName"],
                'AdvancedFilter.ChildObjectsToInclude': ["TkprSchool"],}
response = session.get(timekeeper_url, params=query_params)
timekeepers = response.json(object_hook=lambda d: SimpleNamespace(**d)).timekeepers

In [ ]:
timekeepers

## Updating the timekeeper record
To update a record, the `Patch` method is used. The tricky part here was trying to figure out what the body of the request should look like. There doesn't appear to be any documentation available at the moment for how to do this, but with some trial and error and attempts to decipher PostMan's suggestions we end up with the following. It would be good as with most API's that the Swagger documentation would more clearly state this format. The Postman suggestions are filled with `<Error: Too many levels of nesting to fake this schema>` which is unfortunate. 

It is not clear if the reponse can be tailored to only return specific fields. The response is quite a bit different than the others in that looks like the following.

```json
{
    "success": true,
    "dataCollection": {
        "id": "Timekeeper",
        "objectId": "Timekeeper",
        "actualRowCount": 2,
        "rows": [
            {
                ...
            }
        ]
     }
}            
```

If one of the records is locked by another user, you'll get a `202 Multi-Status` response with the following in the `rows` array/list.
```json
"index": 1,
"isLocked": true,
"lockedMessage": "Timekeeper record was modified by _named_user_ since it was opened.  Reopen the record to save changes.",
"subclassId": "Timekeeper"
```

In [ ]:
# in practice, this would be built up programatically, but for illustrative purposes, this is 
# what the body of the request will contain. The following updates two different timekeeper records in one request.
# the id's will need to be changed to reflect your own instance. The id's can be found in the previous step, or
# by examining the `Timekeeper.TimekeeperID` field using SQL. It is unclear if other identifiers (like TkprIndex, 
# or Number) can be used in place of "id"

request_body = """
{
    "DataCollection": {
        "rows": [
            {
                "id": "7a0c32275e384c2b887b3af476bd1eb5",
                "attributes": {
                    "displayName": {
                        "value": "A MacGillivary "
                    },
                    "SortName": {
                        "value": "MacGillivary, A"
                    }
                }
            },
            {
                "id": "13e3ee7b60694c7583b36ca37eb8ea80",
                "attributes": {
                    "SortName": {
                        "value": "Costanza, G"
                    }

                }
            }
       ]
    }
}
"""

In [ ]:
# in this example, that request_body is a string, it needs to be converted to json when passed to the json argument - use `json.loads` for that
response = session.patch(timekeeper_url, json=json.loads(request_body))
timekeepers_updated = response.json(object_hook=lambda d: SimpleNamespace(**d))

In [ ]:
timekeepers_updated